# Deep Unsupervised Learning course, HSE, fall-winter 2019
## HW 01
### Student: Igor Tukh

# <span style="color:red"> Note: task isn't complete yet. I will finish it a little bit later.</span>

In [20]:
import numpy as np
import torch
import os
import pickle
import torch.optim
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [19]:
RESOURCES_PATH = './resources'
DATA_PATH = os.path.join(RESOURCES_PATH, 'mnist-hw1.pkl')

In [21]:
def load_data():
    with open(DATA_PATH, 'rb') as data_file:
        data = pickle.load(data_file)
    
    return data

In [22]:
data = load_data()

## 3) High-dimensional data

In [11]:
class MaskedConv2d(torch.nn.Conv2d):
    def __init__(self, in_channels, out_channels, 
                 kernel_size, stride=1, padding=0, 
                 dilation=1, groups=1, bias=True, 
                 padding_mode='zeros'):
        super(MaskedConv2d, self).__init__(mask_type, in_channels, out_channels, 
                                           kernel_size, stride, padding,
                                           dilation, groups, bias, 
                                           padding_mode)
        
        self.register_buffer('mask', self.weight.data.clone())
        _, _, kH, kW = self.weight.size()
        self.mask.fill_(1)
        self.mask[:, :, kH // 2, kW // 2 + (mask_type == 'B'):] = 0
        self.mask[:, :, kH // 2 + 1:] = 0

    def forward(self, x):
        self.weight.data *= self.mask
        return super(MaskedConv2d, self).forward(x)

In [15]:
class MaskedResBlock(torch.nn.Module):
    def __init__(self, size):
        self.model = torch.nn.Sequential(
            torch.nn.ReLU(True),
            MaskedConv2d('B', size, size, kernel_size=7, 
                         stride=1, padding=3, bias=False),
            torch.nn.BatchNorm2d(size), 
            torch.nn.ReLU(True),
            MaskedConv2d('B', size, size, kernel_size=7, 
                         stride=1, padding=3, bias=False), 
            torch.nn.BatchNorm2d(size), 
            torch.nn.ReLU(True),
            MaskedConv2d('B', size, size, kernel_size=7, stride=1, 
                         padding=3, bias=False),
            torch.nn.BatchNorm2d(size))
    
    def forward(self, x):
        return self.model(x) + x
    
    def parameters(self):
        return self.model.parameters()

In [16]:
class PixelCNN(torch.nn.Module):
    def __init__(self):
        pass